In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
%matplotlib widget

In [4]:
from trsm import TRSM, combos_6j

In [5]:
import awkward as ak

In [6]:
filename = f'../../NMSSM_XYH_YToHH_6b_MX_700_MY_400_6jet_testing_set.root'
with open("../../testing_event_list.txt") as f:
    list_of_files = f.readlines()
trsm = TRSM(filename=filename)

In [7]:
combos7 = combos_6j(trsm, 7)

100%|█████████████████████████████████████████████████████| 195942/195942 [01:26<00:00, 2263.86it/s]


Total events chosen: 22015


In [8]:
combos7.HX_b1_p4[combos7.combo_mask]

<MomentumArray4D [{pt: 198, eta: -0.846, ... m: 17.1}] type='22015 * Momentum4D[...'>

In [9]:
HX_b1_mask = ak.to_numpy(ak.flatten(combos7.swapped_out_H == 0))
HX_b2_mask = ak.to_numpy(ak.flatten(combos7.swapped_out_H == 1))
H1_b1_mask = ak.to_numpy(ak.flatten(combos7.swapped_out_H == 2))
H1_b2_mask = ak.to_numpy(ak.flatten(combos7.swapped_out_H == 3))
H2_b1_mask = ak.to_numpy(ak.flatten(combos7.swapped_out_H == 4))
H2_b2_mask = ak.to_numpy(ak.flatten(combos7.swapped_out_H == 5))

In [10]:
from keras.models import model_from_json
from pickle import load

In [11]:
tag = 'evans_preselections_2HL_v3_fixed-bug-no-6-jet-requirement'

In [12]:
json_file = open(f'../models/{tag}/model/model_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(f'../models/{tag}/model/model_1.h5')
scaler = load(open(f'../models/{tag}/model/scaler_1.pkl', 'rb'))

2021-08-06 06:51:44.545677: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
test_features_7 = scaler.transform(combos7.combo_features)
scores_7 = loaded_model.predict(test_features_7)[:,0]
scores_7.shape

2021-08-06 06:51:45.869037: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 18492600 exceeds 10% of free system memory.
2021-08-06 06:51:45.923254: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-06 06:51:45.923924: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2000000000 Hz


(154105,)

In [14]:
sgnl_scores_7 = scores_7[combos7.sgnl_mask]
bkgd_scores_7 = scores_7[~combos7.sgnl_mask]

In [15]:
import awkward as ak
import numpy as np

In [16]:
combo_mask = combos7.combo_mask
HX_p4 = ak.to_numpy(combos7.HX_p4.m)[combo_mask]
H1_p4 = ak.to_numpy(combos7.H1_p4.m)[combo_mask]
H2_p4 = ak.to_numpy(combos7.H2_p4.m)[combo_mask]

correct_H = np.concatenate((HX_p4, H1_p4, H2_p4))
incorrect_H = ak.to_numpy(ak.flatten(combos7.incorrect_H_p4.m))

correct_H = correct_H.reshape(correct_H.shape[0],)
incorrect_H = incorrect_H.reshape(incorrect_H.shape[0],)

In [17]:
combos7.combo_features[0]

array([ 1.98343857e+02,  1.62190399e+02,  1.32899078e+02,  6.96733246e+01,
        5.23794975e+01,  3.73367653e+01, -8.46313477e-01, -1.21606445e+00,
       -4.98962402e-01, -2.43438721e-01, -1.28173828e+00,  2.14721680e-01,
       -2.23779297e+00, -4.73022461e-02,  4.04724121e-01, -2.96044922e+00,
       -1.38330078e+00,  1.54638672e+00,  9.76562500e-01,  4.38476562e-01,
        9.82910156e-01,  9.83886719e-01,  8.54980469e-01,  9.99511719e-01,
        2.35930645e+02,  2.38453363e+02,  1.49813705e+02,  6.35504148e+01,
        8.47027080e+01,  3.13647469e+01])

In [18]:
import matplotlib.pyplot as plt
from consistent_plots import hist, hist2d

In [19]:
def norm_hist(arr, bins=100):
    n, b = np.histogram(arr, bins=bins)
    x = (b[:-1] + b[1:]) / 2
    
    return n/n.max(), b, x

In [20]:
fig, ax = plt.subplots()

ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_1, b7, x7 = norm_hist(incorrect_H, bins=np.linspace(0,1000,100))
n7_2, b7, x7 = norm_hist(correct_H, bins=np.linspace(0,1000,100))

hist(ax, x7, weights=n7_1, bins=b7, label='Incorrect')
hist(ax, x7, weights=n7_2, bins=b7, label='Correct')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Dijet Invariant Mass [GeV]')
ax.set_ylabel('AU')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,4))

ax = axs[0]
ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_0, b7, x7 = norm_hist(incorrect_H[HX_b1_mask], bins=np.linspace(0,600,100))
n7_1, b7, x7 = norm_hist(incorrect_H[HX_b2_mask], bins=np.linspace(0,600,100))
n7_2, b7, x7 = norm_hist(HX_p4, bins=np.linspace(0,600,100))

hist(ax, x7, weights=n7_0, bins=b7, label='HX b1')
hist(ax, x7, weights=n7_1, bins=b7, label='HX b2')
hist(ax, x7, weights=n7_2, bins=b7, label='True HX')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Dijet Invariant Mass [GeV]')
ax.set_ylabel('AU')

ax = axs[1]
ax.set_title(r"$^7C_6$ Combinations Non-Normalized")

n, *_ = hist(ax, incorrect_H[HX_b1_mask], bins=b7, label='HX b1')
n, *_ = hist(ax, incorrect_H[HX_b2_mask], bins=b7, label='HX b2')
n, *_ = hist(ax, HX_p4, bins=b7, label='True HX')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Dijet Invariant Mass [GeV]')
ax.set_ylabel('Count')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,4))

ax = axs[0]
ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_0, b7, x7 = norm_hist(incorrect_H[H1_b1_mask], bins=np.linspace(0,600,100))
n7_1, b7, x7 = norm_hist(incorrect_H[H1_b2_mask], bins=np.linspace(0,600,100))
n7_2, b7, x7 = norm_hist(H1_p4, bins=np.linspace(0,600,100))

hist(ax, x7, weights=n7_0, bins=b7, label='H1 b1')
hist(ax, x7, weights=n7_1, bins=b7, label='H1 b2')
hist(ax, x7, weights=n7_2, bins=b7, label='True H1')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Dijet Invariant Mass [GeV]')
ax.set_ylabel('AU')


ax = axs[1]
ax.set_title(r"$^7C_6$ Combinations Non-Normalized")

n, *_ = hist(ax, incorrect_H[H1_b1_mask], bins=b7, label='H1 b1')
n, *_ = hist(ax, incorrect_H[H1_b2_mask], bins=b7, label='H1 b2')
n, *_ = hist(ax, H1_p4, bins=b7, label='True HX')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Dijet Invariant Mass [GeV]')
ax.set_ylabel('Count')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,4))

ax = axs[0]
ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_0, b7, x7 = norm_hist(incorrect_H[H2_b1_mask], bins=np.linspace(0,600,100))
n7_1, b7, x7 = norm_hist(incorrect_H[H2_b2_mask], bins=np.linspace(0,600,100))
n7_2, b7, x7 = norm_hist(H2_p4, bins=np.linspace(0,600,100))

hist(ax, x7, weights=n7_0, bins=b7, label='H2 b1')
hist(ax, x7, weights=n7_1, bins=b7, label='H2 b2')
hist(ax, x7, weights=n7_2, bins=b7, label='True H2')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Dijet Invariant Mass [GeV]')
ax.set_ylabel('AU')

ax = axs[1]
ax.set_title(r"$^7C_6$ Combinations Non-Normalized")

hist(ax, incorrect_H[H2_b1_mask], bins=b7, label='H2 b1')
hist(ax, incorrect_H[H2_b2_mask], bins=b7, label='H2 b2')
hist(ax, H2_p4, bins=b7, label='True H2')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Dijet Invariant Mass [GeV]')
ax.set_ylabel('Count')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,3))

ax = axs[0]
ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_1, b7, x7 = norm_hist(incorrect_H[scores_7[~combos7.sgnl_mask] <= 0.8], bins=np.linspace(0,1000,100))
n7_2, b7, x7 = norm_hist(incorrect_H[scores_7[~combos7.sgnl_mask] > 0.8], bins=np.linspace(0,1000,100))

hist(ax, x7, weights=n7_1, bins=b7, label='Low score')
hist(ax, x7, weights=n7_2, bins=b7, label='High Score')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel('Incorrect Higgs mass')
ax.set_ylabel('AU')

ax = axs[1]
ax.set_title(r"$^7C_6$ Combinations")

n7_1, b7 = np.histogram(incorrect_H[scores_7[~combos7.sgnl_mask] <= 0.8], bins=np.linspace(0,1000,100))
n7_2, b7 = np.histogram(incorrect_H[scores_7[~combos7.sgnl_mask] > 0.8], bins=np.linspace(0,1000,100))

hist(ax, x7, weights=n7_1, bins=b7, label='Low score')
hist(ax, x7, weights=n7_2, bins=b7, label='High Score')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel('Incorrect Higgs mass')
ax.set_ylabel('AU')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10,10))

ax = axs[0][0]
n, x, y, im = hist2d(ax, incorrect_H, scores_7[~combos7.sgnl_mask], xbins=np.linspace(0,500,100))
plt.colorbar(im, ax=ax)
ax.set_ylabel("Assigned Score")

ax = axs[0][1]
n, x, y, im = hist2d(ax, incorrect_H[HX_b1_mask | HX_b2_mask], scores_7[~combos7.sgnl_mask][HX_b1_mask | HX_b2_mask], xbins=np.linspace(0,500,100))
plt.colorbar(im, ax=ax)
ax.set_ylabel("Assigned Score")

ax = axs[1][0]
n, x, y, im = hist2d(ax, incorrect_H[H1_b1_mask | H1_b2_mask], scores_7[~combos7.sgnl_mask][H1_b1_mask | H1_b2_mask], xbins=np.linspace(0,500,100))
plt.colorbar(im, ax=ax)
ax.set_ylabel("Assigned Score")

ax = axs[1][1]
n, x, y, im = hist2d(ax, incorrect_H[H2_b1_mask | H2_b2_mask], scores_7[~combos7.sgnl_mask][H2_b1_mask | H2_b2_mask], xbins=np.linspace(0,500,100))
plt.colorbar(im, ax=ax)
ax.set_ylabel("Assigned Score")

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
fig, ax = plt.subplots(nrows=2, ncols=3)

n, x, y, im = hist2d(ax, incorrect_H, scores_7[~combos7.sgnl_mask], xbins=np.linspace(0,500,100))
plt.colorbar(im, ax=ax)

ax.set_ylabel("Assigned Score")

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AttributeError: 'numpy.ndarray' object has no attribute 'hist2d'

In [ ]:
fig, ax = plt.subplots()

ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_0, b7, x7 = norm_hist(bkgd_scores_7[HX_b1_mask], bins=np.linspace(0,1.01,100))
n7_1, b7, x7 = norm_hist(bkgd_scores_7[HX_b2_mask], bins=np.linspace(0,1.01,100))

hist(ax, x7, weights=n7_0, bins=b7, label='HX b1')
hist(ax, x7, weights=n7_1, bins=b7, label='HX b2')
# hist(ax, x7, weights=n7_2, bins=b7, label='True H2')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Assigned Score')
ax.set_ylabel('AU')

plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_0, b7, x7 = norm_hist(bkgd_scores_7[H1_b1_mask], bins=np.linspace(0,1.01,100))
n7_1, b7, x7 = norm_hist(bkgd_scores_7[H1_b2_mask], bins=np.linspace(0,1.01,100))

hist(ax, x7, weights=n7_0, bins=b7, label='H1 b1')
hist(ax, x7, weights=n7_1, bins=b7, label='H1 b2')
# hist(ax, x7, weights=n7_2, bins=b7, label='True H2')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Assigned Score')
ax.set_ylabel('AU')

plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_0, b7, x7 = norm_hist(bkgd_scores_7[H2_b1_mask], bins=np.linspace(0,1.01,100))
n7_1, b7, x7 = norm_hist(bkgd_scores_7[H2_b2_mask], bins=np.linspace(0,1.01,100))

hist(ax, x7, weights=n7_0, bins=b7, label='H2 b1')
hist(ax, x7, weights=n7_1, bins=b7, label='H2 b2')
# hist(ax, x7, weights=n7_2, bins=b7, label='True H2')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Assigned Score')
ax.set_ylabel('AU')

plt.show()